In [1]:
import os
os.environ["CUDA_VISIBLE_DEVICES"]="0"
import numpy as np
import pandas as pd
import pickle
from keras_bert import load_trained_model_from_checkpoint, Tokenizer,get_custom_objects
import codecs
from keras.layers import Input,Dense
from keras.models import Model,load_model
from keras.optimizers import Adam

In [2]:
from tensorflow.python.client import device_lib
print(device_lib.list_local_devices())

[name: "/device:CPU:0"
device_type: "CPU"
memory_limit: 268435456
locality {
}
incarnation: 7681536765323704628
, name: "/device:GPU:0"
device_type: "GPU"
memory_limit: 3127299276
locality {
  bus_id: 1
  links {
  }
}
incarnation: 18359831461309078996
physical_device_desc: "device: 0, name: GeForce GTX 1050 Ti, pci bus id: 0000:01:00.0, compute capability: 6.1"
]


In [2]:
max_seq_len = 20
config_path = r'D:\final_design\Final_one\bert\bert_config.json'
checkpoint_path = r'D:\final_design\Final_one\bert\bert_model.ckpt'
dict_path = r'D:\final_design\Final_one\bert\vocab.txt'

corpus_path = r'C:\Users\HashiriNio\Desktop\lcqmc-NLP\lcqmc_train.tsv'

In [3]:
token_dict = {}
with codecs.open(dict_path, 'r', 'utf8') as reader:
    for line in reader:
        token = line.strip()
        token_dict[token] = len(token_dict)
tokenizer = Tokenizer(token_dict)

In [4]:
trainx1 = []
trainx2 = []
trainy = []
with codecs.open(corpus_path, 'r', 'utf8') as reader:
    for i in reader:
        trainx1.append(i.split('\t')[0])
        trainx2.append(i.split('\t')[1])
        trainy.append(i.split('\t')[2].strip())

In [5]:
trainx1 = trainx1[1:]
trainx2 = trainx2[1:]
trainy = trainy[1:]

In [6]:
def get_tokens(seq1,seq2):
    X1_1,X1_2,X2_1,X2_2 = [] ,[],[],[]
    for i in range(len(seq1)):
        x1_1,x1_2 = tokenizer.encode(first = seq1[i],max_len = max_seq_len)
        x2_1,x2_2 = tokenizer.encode(first = seq2[i],max_len = max_seq_len)
        X1_1.append(x1_1)
        X1_2.append(x1_2)
        X2_1.append(x2_1)
        X2_2.append(x2_2)
    return np.array(X1_1), np.array(X1_2), np.array(X2_1), np.array(X2_2)

In [7]:
train1_1,train1_2,train2_1,train2_2 = get_tokens(trainx1,trainx2)

In [8]:
import tensorflow as tf

In [29]:
#def get_lr_metric(optimizer):  # printing the value of the learning rate
#    def lr(y_true, y_pred):
#        return optimizer.lr
#    return lr

#optimizer = Adam(lr=1e-4)  
#lr_metric = get_lr_metric(optimizer)


bert_model = load_trained_model_from_checkpoint(config_path, checkpoint_path, seq_len=None)
drop_out_rate = 0.5
dense_layer = 512
class_num = 2
#for l in bert_model.layers:
#    l.trainable = True

x1_1_in = Input(shape=(None,))
x1_2_in = Input(shape=(None,))
x2_1_in = Input(shape=(None,))
x2_2_in = Input(shape=(None,))


x1 = bert_model([x1_1_in, x1_2_in])
x2 = bert_model([x2_1_in, x2_2_in])
Lambda= lambda x: x[:, 0]
x1 = Lambda(x1)
x2 = Lambda(x2)

concat_layer = tf.concat((x1,x2,tf.abs(tf.math.subtract(x1, x2))),1)
drop_out = tf.keras.layers.Dropout(drop_out_rate)(concat_layer)
Dense_l = Dense(dense_layer,activation='relu')(drop_out)
output = Dense(class_num,activation='softmax')(Dense_l)

model = Model([x1_1_in,x1_2_in,x2_1_in,x2_2_in], output)
model.compile(loss='binary_crossentropy',optimizer=Adam(lr=1e-4), metrics=['accuracy'])
model.summary()
model.save(r'D:\final_design\Final_one\data\model\model_match_general5.h5')


Model: "model_8"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_9 (InputLayer)            [(None, None)]       0                                            
__________________________________________________________________________________________________
input_10 (InputLayer)           [(None, None)]       0                                            
__________________________________________________________________________________________________
input_11 (InputLayer)           [(None, None)]       0                                            
__________________________________________________________________________________________________
input_12 (InputLayer)           [(None, None)]       0                                            
____________________________________________________________________________________________

In [11]:
from keras.utils.generic_utils import get_custom_objects as gt_custom_objects

In [44]:
{'lr_metric': lr_metric,'gelu': <function gelu at 0x0000026AAFCC59D8>, 'LayerNormalization': <class 'keras_layer_normalization.layer_normalization.LayerNormalization'>, 'MultiHeadAttention': <class 'keras_multi_head.multi_head_attention.MultiHeadAttention'>, 'FeedForward': <class 'keras_position_wise_feed_forward.feed_forward.FeedForward'>, 'TrigPosEmbedding': <class 'keras_pos_embd.trig_pos_embd.TrigPosEmbedding'>, 'EmbeddingRet': <class 'keras_embed_sim.embeddings.EmbeddingRet'>, 'EmbeddingSim': <class 'keras_embed_sim.embeddings.EmbeddingSim'>, 'PositionEmbedding': <class 'keras_pos_embd.pos_embd.PositionEmbedding'>, 'TokenEmbedding': <class 'keras_bert.layers.embedding.TokenEmbedding'>, 'EmbeddingSimilarity': <class 'keras_bert.layers.embedding.EmbeddingSimilarity'>, 'TaskEmbedding': <class 'keras_bert.layers.task_embed.TaskEmbedding'>, 'Masked': <class 'keras_bert.layers.masked.Masked'>, 'Extract': <class 'keras_bert.layers.extract.Extract'>, 'AdamWarmup': <class 'keras_bert.optimizers.warmup_v2.AdamWarmup'>}

SyntaxError: invalid syntax (<ipython-input-44-0b53280228a2>, line 1)

In [15]:
gt_custom_objects().update({'lr':lr_metric})
gt_custom_objects().update(get_custom_objects())

In [73]:
gt_custom_objects().clear()

In [16]:
print(gt_custom_objects())

{'lr': <function get_lr_metric.<locals>.lr at 0x000001CE93B53318>, 'gelu': <function gelu at 0x000001CE12DB5AF8>, 'LayerNormalization': <class 'keras_layer_normalization.layer_normalization.LayerNormalization'>, 'MultiHeadAttention': <class 'keras_multi_head.multi_head_attention.MultiHeadAttention'>, 'FeedForward': <class 'keras_position_wise_feed_forward.feed_forward.FeedForward'>, 'TrigPosEmbedding': <class 'keras_pos_embd.trig_pos_embd.TrigPosEmbedding'>, 'EmbeddingRet': <class 'keras_embed_sim.embeddings.EmbeddingRet'>, 'EmbeddingSim': <class 'keras_embed_sim.embeddings.EmbeddingSim'>, 'PositionEmbedding': <class 'keras_pos_embd.pos_embd.PositionEmbedding'>, 'TokenEmbedding': <class 'keras_bert.layers.embedding.TokenEmbedding'>, 'EmbeddingSimilarity': <class 'keras_bert.layers.embedding.EmbeddingSimilarity'>, 'TaskEmbedding': <class 'keras_bert.layers.task_embed.TaskEmbedding'>, 'Masked': <class 'keras_bert.layers.masked.Masked'>, 'Extract': <class 'keras_bert.layers.extract.Extrac

In [9]:
model = load_model(r'D:\final_design\Final_one\data\model\model_match_general5.h5',custom_objects=get_custom_objects())

In [14]:
def get_lr_metric(optimizer):  # printing the value of the learning rate
    def lr(y_true, y_pred):
        return optimizer.lr
    return lr

optimizer = Adam(lr=1e-4)  
lr_metric = get_lr_metric(optimizer)

In [15]:
model.optimizer.lr

<tf.Variable 'learning_rate:0' shape=() dtype=float32, numpy=6.25e-06>

In [16]:
from keras.callbacks import ReduceLROnPlateau

In [19]:
reduce_lr = ReduceLROnPlateau(monitor='val_loss',factor=0.5, patience=6, mode='min')

In [22]:
p = 60000
p2 = p+30000
#epoch = 4

model.fit([train1_1[p:p2],train1_2[p:p2],train2_1[p:p2],train2_2[p:p2]],trainy[p:p2], epochs=10, batch_size=64,validation_data = ([train1_1[200000:201000],train1_2[200000:201000],train2_1[200000:201000],train2_2[200000:201000]],trainy[200000:201000]),callbacks=[reduce_lr],shuffle=True)
print(model.optimizer.lr)

Epoch 1/10
469/469 [==============================] - 213s 455ms/step - loss: 0.5093 - accuracy: 0.7531 - val_loss: 0.4530 - val_accuracy: 0.7950
Epoch 2/10
469/469 [==============================] - 217s 462ms/step - loss: 0.5106 - accuracy: 0.7538 - val_loss: 0.4549 - val_accuracy: 0.7870
Epoch 3/10
469/469 [==============================] - 219s 468ms/step - loss: 0.5116 - accuracy: 0.7513 - val_loss: 0.4533 - val_accuracy: 0.7930
Epoch 4/10
469/469 [==============================] - 219s 468ms/step - loss: 0.5112 - accuracy: 0.7539 - val_loss: 0.4539 - val_accuracy: 0.7890
Epoch 5/10
469/469 [==============================] - 219s 466ms/step - loss: 0.5107 - accuracy: 0.7505 - val_loss: 0.4557 - val_accuracy: 0.7870
Epoch 6/10
469/469 [==============================] - 218s 465ms/step - loss: 0.5096 - accuracy: 0.7551 - val_loss: 0.4548 - val_accuracy: 0.7900
Epoch 7/10
469/469 [==============================] - 218s 465ms/step - loss: 0.5086 - accuracy: 0.7498 - val_loss: 0.4567 -

In [23]:
model.save(r'D:\final_design\Final_one\data\model\model_match_general5_1.h5')

In [ ]:
#loss: 0.5221 - accuracy: 0.7422 - val_loss: 0.4793 - val_accuracy: 0.7850 
#loss: 0.4933 - accuracy: 0.7630 - val_loss: 0.4689 - val_accuracy: 0.7880
# loss: 0.5147 - accuracy: 0.7512 - val_loss: 0.4606 - val_accuracy: 0.7940
#loss: 0.5179 - accuracy: 0.7468 - val_loss: 0.4564 - val_accuracy: 0.7900
#loss: 0.5074 - accuracy: 0.7526 - val_loss: 0.4558 - val_accuracy: 0.7900

#（1）val_loss: 0.5682 - val_accuracy: 0.7600    （2)val_loss: 0.5611 - val_accuracy: 0.7680 
#(3)val_loss: 0.4719 - val_accuracy: 0.7850       (4)

In [ ]:
m1 = 2000
m2 = 4000
#loss: 0.7122 - accuracy: 0.5740
#loss: 0.6911 - accuracy: 0.5997
#loss: 0.6421 - accuracy: 0.6437
#loss: 0.6079 - accuracy: 0.6761
#loss: 0.6023 - accuracy: 0.6830
#loss: 0.5837 - accuracy: 0.6894
#loss: 0.5724 - accuracy: 0.7027
#loss: 0.5606 - accuracy: 0.7077
#loss: 0.5576 - accuracy: 0.7178
#loss: 0.5492 - accuracy: 0.7239  (56000:66000
#loss: 0.5347 - accuracy: 0.7272
#loss: 0.5191 - accuracy: 0.7458

In [10]:
trainy = [int(x) for x in trainy]


In [11]:
trainy = [[0,1] if x == 0 else [1,0] for x in trainy]

In [12]:
trainy = np.array(trainy)

In [17]:
trainy[:10]

array([[1, 0],
       [1, 0],
       [0, 1],
       [1, 0],
       [0, 1],
       [1, 0],
       [0, 1],
       [1, 0],
       [1, 0],
       [1, 0]])

In [42]:
trainy[:10]

array([1, 1, 0, 1, 0, 1, 0, 1, 1, 1])

In [27]:
#len(train1_1)
#8000（0.5960  
#训练进程：2000:4000

238766

In [41]:
#from keras.layers.core import Lambda
#'''
bert_model = load_trained_model_from_checkpoint(config_path, checkpoint_path, seq_len=None)
drop_out_rate = 0.2
dense_layer = 512
class_num = 1
#for l in bert_model.layers:
#    l.trainable = True

x1_1_in = Input(shape=(None,))
x1_2_in = Input(shape=(None,))
x2_1_in = Input(shape=(None,))
x2_2_in = Input(shape=(None,))


x1 = bert_model([x1_1_in, x1_2_in])
x2 = bert_model([x2_1_in, x2_2_in])
Lambda= lambda x: x[:, 0]
x1 = Lambda(x1)
x2 = Lambda(x2)
#print(x1[0].shape)

dropl = tf.keras.layers.Dropout(drop_out_rate)
drop_out_1 = dropl(x1)
drop_out_2 = dropl(x2)
#Dense_l = Dense(dense_layer,activation='relu')
#m1 = Dense_l(drop_out_1)
#m2 = Dense_l(drop_out_2)
#cosinelayer = CosineLayer()
#intern = cosinelayer(x1,x2)
output = Dense(class_num,activation='sigmoid')(intern)

model = Model([x1_1_in,x1_2_in,x2_1_in,x2_2_in], output)
model.compile(loss='binary_crossentropy',optimizer=Adam(1e-5),metrics=['accuracy'])
model.summary()
model.save(r'D:\final_design\Final_one\data\model\model_match_general_2.h5')
#'''

Model: "model_14"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_17 (InputLayer)           [(None, None)]       0                                            
__________________________________________________________________________________________________
input_18 (InputLayer)           [(None, None)]       0                                            
__________________________________________________________________________________________________
input_19 (InputLayer)           [(None, None)]       0                                            
__________________________________________________________________________________________________
input_20 (InputLayer)           [(None, None)]       0                                            
___________________________________________________________________________________________

In [4]:
import tensorflow as tf

In [11]:
from tensorflow import keras


In [17]:
tf.__version__

'2.4.1'

In [1]:
import tensorflow as tf
tf.test.is_gpu_available(
        cuda_only=False,
        min_cuda_compute_capability=None
    )

Instructions for updating:
Use `tf.config.list_physical_devices('GPU')` instead.


True

In [10]:
text = '语言模型'
tokens = tokenizer.tokenize(text)
# ['[CLS]', '语', '言', '模', '型', '[SEP]']
indices, segments = tokenizer.encode(first=text, max_len=512)
print(indices[:10])
print(segments[:10])

model = load_trained_model_from_checkpoint(config_path, checkpoint_path)
predicts = model.predict([np.array([indices]), np.array([segments])])

[101, 6427, 6241, 3563, 1798, 102, 0, 0, 0, 0]
[0, 0, 0, 0, 0, 0, 0, 0, 0, 0]


In [13]:
type(predicts)


numpy.ndarray

In [23]:
Lambda= lambda x: x[:, 1]
x1 = Lambda(predicts)
x1.shape

(1, 768)

In [35]:
import keras.backend as K
#from keras.layers.core import Lambda

class CosineLayer():

    def __call__(self, x1, x2):

        def _cosine(x):
            dot1 = K.batch_dot(x[0], x[1], axes=1)
            dot2 = K.batch_dot(x[0], x[0], axes=1)
            dot3 = K.batch_dot(x[1], x[1], axes=1)
            max_ = K.maximum(K.sqrt(dot2 * dot3), K.epsilon())
            return dot1 / max_

        #output_shape = (1,)
        Lambda = lambda x :_cosine(x)
        value = Lambda([x1, x2])
        return value


In [44]:
x1 = np.array([1,2,3,4,5,6])
x2 = np.array([5,1,3,4,8,6])
k = CosineLayer()
m = k(x1,x2)

ValueError: Cannot do batch_dot on inputs with rank < 2. Received inputs with shapes (6,) and (6,).

In [40]:
m

0.921328669759435

In [1]:
2%2

0

In [4]:
i = 5
path = r'D:\final_design\Final_one\data\model\model_match_general%d.h5'%i

In [5]:
path

'D:\\final_design\\Final_one\\data\\model\\model_match_general5.h5'

In [31]:
help()


Welcome to Python 3.7's help utility!

If this is your first time using Python, you should definitely check out
the tutorial on the Internet at https://docs.python.org/3.7/tutorial/.

Enter the name of any module, keyword, or topic to get help on writing
Python programs and using Python modules.  To quit this help utility and
return to the interpreter, just type "quit".

To get a list of available modules, keywords, symbols, or topics, type
"modules", "keywords", "symbols", or "topics".  Each module also comes
with a one-line summary of what it does; to list the modules whose name
or summary contain a given string such as "spam", type "modules spam".

help> keras_bert.get_custom_objects
Help on function get_custom_objects in keras_bert:

keras_bert.get_custom_objects = get_custom_objects()
    Get all custom objects for loading saved models.

help> quit

You are now leaving help and returning to the Python interpreter.
If you want to ask for help on a particular object directly from t